In [15]:
import pandas as pd
import numpy as np
import vnstock as vn
import calculation as cal

In [16]:
data = cal.DataProcessor.load_data('VCB') 
data = data.set_index('time')
#Tính trung bình 20 phiên gần nhất
mean_20 = data['volume'].rolling(window=20).mean() 
#Tính on-balance volume
data['on-balance_volume'] = data['volume']/mean_20 

data['OBV_label'] = data['on-balance_volume'].apply(cal.DataProcessor.label_values)
#Giá đóng cửa ngày t-1
data['close_t_minus_1'] = data['close'].shift(1) 
data['bar_type'] = data.apply(cal.DataProcessor.compare_close_prices, axis=1)   
data['para'] =abs(data['close'] - data['open'])/(data['high'] - data['low'])

data['label_spread'] = data['para'].apply(cal.DataProcessor.label_spread)
data['close_bar_label'] = data.apply(cal.DataProcessor.label_close_bar, axis=1)


In [17]:
data.head()

,open,high,low,close,volume,ticker,on-balance_volume,OBV_label,close_t_minus_1,bar_type,para,label_spread,close_bar_label
time,,,,,,,,,,,,,
2019-01-02,34750,35010,34490,34560,1081640,VCB,NaN,NaN,NaN,down-bar,0.365385,medium,bottom-third
2019-01-03,34820,34880,33980,34560,1071350,VCB,NaN,NaN,34560.0,down-bar,0.288889,low,middle-third
2019-01-04,34370,35080,33910,35080,1307310,VCB,NaN,NaN,34560.0,up-bar,0.606838,medium,top-third
2019-01-07,35660,35780,35140,35200,1175810,VCB,NaN,NaN,35080.0,up-bar,0.718750,high,bottom-third
2019-01-08,35460,35590,35140,35460,1318810,VCB,NaN,NaN,35200.0,up-bar,0.000000,high,top-third


## Weakness A
- Down bar
- High volume
- Medium or low spread
- Closing in the middle-third or  bottom-third of the bar

In [18]:
def is_weakness_a_signal(data):
    if (data['bar_type'] == 'down-bar') and \
        (data['OBV_label']=='high') and \
        ((data['label_spread'] == 'low') or (data['label_spread']=='medium')) and \
        ((data['close_bar_label'] == 'middle-third') or (data['close_bar_label']=='bottom-third')):
        return True
    else:
        return False

In [19]:
# Sample data
sample_data = {
    'bar_type': 'down-bar',
    'OBV_label': 'high',
    'label_spread': 'low',
    'close_bar_label': 'middle-third'
}
result = is_weakness_a_signal(sample_data)
print("Is weakness a signal?", result)

Is weakness a signal? True


## No demand
- up bar
- volume: low or high
- spread: low or high
- close: bottom-third

In [20]:
def is_nodemand_signal(data):
    if (data['bar_type'] == 'up-bar') and \
        ((data['OBV_label']=='high') or (data['OBV_label']=='low')) and \
        ((data['label_spread'] == 'low') or (data['label_spread']=='high')) and \
        ((data['close_bar_label'] == 'bottom-third') ):
        return True
    else:
        return False

In [21]:
result = is_nodemand_signal(sample_data)
print("Is no demand signal?", result)

Is no demand signal? False


## Up trust
- Up bar or down bar (This bar must update the local maximum and its maximum should not be redrawn)
- Spread high
- The closing should be in the lower third
- Volume is high or very high


In [22]:
def is_up_trust_signal(data):
    if ((data['OBV_label']=='high') or (data['OBV_label']=='low')) and \
        (data['label_spread'] == 'high') and \
        ((data['close_bar_label'] == 'bottom-third') ):
        return True
    else:
        return False

## Stop volume
- up bar
- high spread
- high volume

In [23]:
def is_stop_volume_signal(data):
    if (data['bar_type']=='up-bar') and \
        (data['OBV_label']=='high') and \
        (data['label_spread'] == 'high'):
        return True
    else:
        return False